In [ ]:
## TAAS 교통사고데이터 좌표 크롤러 with Selenium
Selenium을 이용한 동적 크롤링

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from urllib import parse
import time
import pandas as pd
import re
from tqdm import tqdm

In [32]:
# TAAS GIS 시스템 로드
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('http://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')
time.sleep(1)

In [34]:
#융합분석(menuRoadNoSearch) 선택
menuRoadNoSearch_element = driver.find_element(By.ID, 'menuRoadNoSearch')

# menuRoadNoSearch를 찾았다면 클릭
if menuRoadNoSearch_element:
    menuRoadNoSearch_element.click()


# selectBox지정
#2018년 시작
selectboxStart = driver.find_element(By.ID, 'ptsNafYearStart')
select = Select(selectboxStart)
select.select_by_value("2018") 

#2020년 종료
selectboxYearEnd = driver.find_element(By.ID, 'ptsNafYearEnd')
select = Select(selectboxYearEnd)
select.select_by_value("2020") 

#고속국도 조회
selectboxRoadRank = driver.find_element(By.ID, 'ptsNafRoadRank')
select = Select(selectboxRoadRank)
select.select_by_visible_text("고속국도") 

#도로명 지정
selectboxRoadName = driver.find_element(By.ID, 'ptsNafCh1RoadName')
select = Select(selectboxRoadName)
select.select_by_visible_text("구마고속도로") 



#checkbox 지정
#조건설정 전부 체크
checkbox_elements = driver.find_elements(By.CSS_SELECTOR, '#ptsNafCh1AccidentContent input[type="checkbox"]')

# Check all checkboxes
for checkbox in checkbox_elements:
    if not checkbox.is_selected():
        checkbox.click()



In [6]:
# search
driver.find_element(By.XPATH, '//*[@id="subRoadNumberAccidentFind"]/div[2]/p/a').send_keys(Keys.ENTER)
time.sleep(1)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="subRoadNumberAccidentFind"]/div[2]/p/a"}
  (Session info: chrome=120.0.6099.225); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6B7502142+3514994]
	(No symbol) [0x00007FF6B7120CE2]
	(No symbol) [0x00007FF6B6FC76AA]
	(No symbol) [0x00007FF6B7011860]
	(No symbol) [0x00007FF6B701197C]
	(No symbol) [0x00007FF6B7054EE7]
	(No symbol) [0x00007FF6B703602F]
	(No symbol) [0x00007FF6B70528F6]
	(No symbol) [0x00007FF6B7035D93]
	(No symbol) [0x00007FF6B7004BDC]
	(No symbol) [0x00007FF6B7005C64]
	GetHandleVerifier [0x00007FF6B752E16B+3695259]
	GetHandleVerifier [0x00007FF6B7586737+4057191]
	GetHandleVerifier [0x00007FF6B757E4E3+4023827]
	GetHandleVerifier [0x00007FF6B72504F9+689705]
	(No symbol) [0x00007FF6B712C048]
	(No symbol) [0x00007FF6B7128044]

In [35]:
coord_ls = []
srs_ls = []

for i in tqdm(range(0, 1)):
    driver.execute_script(f'gis.srh.msh.selectListUi({i}, 11);')  # 개별 사고지점
    time.sleep(1)  # 로드
    bbox = driver.find_elements(By.CLASS_NAME, 'olTileImage')[-1]

    src = parse.unquote(bbox.get_attribute('src'))  # get src and unquote from 16byte
    print("src : 얘 출력해줘")

    # 디버깅 코드
    if isinstance(src, list):
        print("Debugging - src content:")
        for element in src:
            print(element)
    else:
        print("Debugging - src content:", src)

    src_split = src.split('&')
    print("src_split : " + str(src_split))  # Use str() to convert list to string

    if isinstance(src_split, list):
        print("Debugging - src_split:")
        for element in src_split:
            print(element)
    else:
        print("Debugging - src_split:", src_split)

    bbox_text = src_split[-3][5:]
    print("bbox_text : " + bbox_text)
    # coords = [float(i) for i in bbox_text.split(',')]
    # x, y = (coords[0] + coords[2]) / 2, (coords[1] + coords[3]) / 2  # center position
    # srs = src_split[-4][4:]
    # coord_ls.append([x, y])
    # srs_ls.append(srs)


  0%|          | 0/1 [00:01<?, ?it/s]

src : 얘 출력해줘
Debugging - src content: https://map2.daumcdn.net/map_2d/2212ejo/L14/-3/3.png
src_split : ['https://map2.daumcdn.net/map_2d/2212ejo/L14/-3/3.png']
Debugging - src_split:
https://map2.daumcdn.net/map_2d/2212ejo/L14/-3/3.png


IndexError: list index out of range